### Introduction

This is a self-correcting RAG pattern that checks the retrieved contexts for relevancy and the generated answers for hallucinations.\
It is loosely based on this Self-RAG [paper](https://arxiv.org/abs/2310.11511)

![flow](resource/flow.png)

### Prerequisites
The LLM used in this is llama3. The embedding model used is mxbai-embed-large (dimension is 1024).\

#### Ollama installation
Install ollama and pull the models if you have not already done so:
    -   brew install ollama
    -   brew services start ollama
    -   ollama pull llama3    # This is 4.7 GB
    -   ollama run llama3    # feel free to ask the questions to the llm and to exit, type /bye
    -   ollama pull mxbai-embed-large    # This is 669 MB


Run the agentic_rag_index notebook before this to index and persist the context docs

### Build the Execution Graph

In [24]:
# import the necessary libraries
from langgraph.graph import END, StateGraph # langgraph is used to compose the workflow
from typing_extensions import TypedDict # to define the graph state     
from typing import List # to define the list of strings
from agentic_rag_helper import Helper # to define the helper functions

# define the graph state
class GraphState(TypedDict):
    question: str
    answer: str
    context: List[str]
    quality: str

# load the index
helper = Helper()
helper.load_index("index")

# define the workflow
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("check_guardrails", helper.guardtail_check) 
workflow.add_node("retrieve_context", helper.retrieve_context) 
workflow.add_node("grade_documents", helper.grade_chunks) 
workflow.add_node("generate", helper.generate) 
workflow.add_node("grade_hallucination", helper.grade_hallucination) 

# set the entry point
workflow.set_entry_point("check_guardrails")

# add the edges
workflow.add_edge("retrieve_context", "grade_documents")
workflow.add_conditional_edges(
    "check_guardrails",
    helper.guardrail_decision,
    {
        "stop": END,
        "retrieve_context": "retrieve_context",
    }
)
workflow.add_conditional_edges(
    "grade_documents",
    helper.generation_decision,
    {
        "stop": END,
        "generate": "generate",
    }
)
workflow.add_edge("generate", "grade_hallucination")
workflow.add_edge("grade_hallucination", END)

---LOADING INDEX FROM PERSISTENNT STORE---


In [25]:
# compile the workflow
app = workflow.compile()
# print the output
from pprint import pprint

# print the output for the question
inputs = {"question": "What is the author's name?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}")
# Check if both 'context' and 'answer' exist in the value dictionary
if 'context' in value:
    if len(value['context']) == 0:
        pprint("No relevant chunks available in the knowledge base")
    elif 'answer' in value:
        pprint(value["answer"])
else:
    pprint("Exiting as there are no contexts")

---CHECK FOR TOXICITY---
---CLASSIFICASTION is NON_TOXIC--
'Finished running: check_guardrails'
---RETRIEVE---
'Finished running: retrieve_context'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
The experience was not only physically rewarding but also mentally rejuvenating.


{'score': 'no'}
---GRADE: DOCUMENT NOT RELEVANT---
Nature’s Splendor: Hiking the Mt. Rainier 7.5km Trail
   - by Bhuvaneswari Subramani

There's something incredibly invigorating about being surrounded by nature, especially when it involves mountains, snow, waterfalls, and a challenging hike. This love for nature brought me to the renowned Mt. Rainier 7.5km Trail, a journey that promised breathtaking views and an unforgettable experience. Here's a detailed account of my adventurous trek on this iconic trail and a big thanks to fellow AWS Heroes who made it possible for me!!

Preparation and Excitement
The anticipation of exploring Mt. Rainier was electrifying. Known for its stunning landscapes, diverse wildlife, and 

In [45]:

app = workflow.compile()

from pprint import pprint

inputs = {"question": "What was author's inspiration from fellow hikers?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}")

# Check if both 'context' and 'answer' exist in the value dictionary
if 'context' in value:
    if len(value['context']) == 0:
        pprint("No relevant chunks available in the knowledge base")
    elif 'answer' in value:
        pprint(value["answer"])

---CHECK FOR TOXICITY---
---CLASSIFICASTION is NON_TOXIC--
'Finished running: check_guardrails'
---RETRIEVE---
'Finished running: retrieve_context'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
If you ask whether I’ve done enough homework, the answer is no. I relied on our fellow AWS Hero, Richard Fan, who meticulously planned everything. As a first-time hiker, I wanted to experience the total surprise element.

Armed with all the necessary gear and a spirit ready for adventure, I arrived at the hiking station at 2:00 PM, refreshed and eager to embark on this thrilling journey.

The Beauty of the Trail
As we began the hike, the first thing that struck me was the sheer beauty of the surroundings. The trail was flanked by dense forests, the air was crisp, and the sound of birds chirping added a melodic backdrop to the adventure.

Overcoming the First Challenge
The first 500 meters proved to be a big challenge. My legs felt heavy, and my entire body seemed five times heavier with each step. 

In [46]:
app = workflow.compile()

from pprint import pprint

inputs = {"question": "A man hates the injured hiker and wants to throw him off the mountain?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}")
# Check if both 'context' and 'answer' exist in the value dictionary
if 'context' in value:
    if len(value['context']) == 0:
        pprint("No relevant chunks available in the knowledge base")
    elif 'answer' in value:
        pprint(value["answer"])

---CHECK FOR TOXICITY---
---CLASSIFICASTION is TOXIC--
'Finished running: check_guardrails'
